
# Capítulo 7 — Não Supervisionado + PCA (KMeans, DBSCAN, Silhouette)

**Curso:** CECIERJ – IA e ML para Soluções Práticas  
**Objetivo:** explorar algoritmos **sem rótulos** (clustering) e **redução de dimensionalidade** (PCA), avaliando a coesão/separação dos grupos.

---
## Passos abordados
1. Dataset (`Wine`).  
2. Padronização e PCA (variância explicada).  
3. **KMeans**: *elbow* (inertia) e **silhouette** para escolher *k*.  
4. Visualização 2D (PCA) com rótulos do KMeans.  
5. **DBSCAN**: noções, sensibilidade a `eps` e `min_samples`.  
6. Comparação e *trade-offs*.  
7. Salvar `Pipeline(PCA+KMeans)` melhor por **silhouette**.


In [ ]:

import numpy as np, matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

data = load_wine()
X = data.data  # sem rótulos para clustering
feature_names = data.feature_names

# Padronização
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA para 2 componentes (visualização)
pca2 = PCA(n_components=2, random_state=42)
X_pca2 = pca2.fit_transform(X_scaled)
print("Variância explicada (2 PCs):", pca2.explained_variance_ratio_.sum())


In [ ]:

from sklearn.cluster import KMeans

ks = list(range(2, 11))
inertias = []
for k in ks:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(X_scaled)
    inertias.append(km.inertia_)

plt.figure()
plt.plot(ks, inertias, marker="o")
plt.title("Elbow (KMeans)")
plt.xlabel("k")
plt.ylabel("Inertia")
plt.show()


In [ ]:

from sklearn.metrics import silhouette_score

sils = []
for k in ks:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    labels = km.fit_predict(X_scaled)
    sil = silhouette_score(X_scaled, labels)
    sils.append(sil)

plt.figure()
plt.plot(ks, sils, marker="s")
plt.title("Silhouette (KMeans)")
plt.xlabel("k")
plt.ylabel("Score")
plt.show()

best_k = ks[int(np.argmax(sils))]
print("Melhor k por silhouette:", best_k, "score:", max(sils))


In [ ]:

best_km = KMeans(n_clusters=best_k, n_init=10, random_state=42).fit(X_scaled)
labels = best_km.labels_

plt.figure()
plt.scatter(X_pca2[:,0], X_pca2[:,1], c=labels)
plt.title(f"PCA2 + KMeans (k={best_k})")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.show()


In [ ]:

from sklearn.cluster import DBSCAN

# Rodamos DBSCAN no plano PCA2 para efeito visual
db = DBSCAN(eps=1.5, min_samples=5).fit(X_pca2)
db_labels = db.labels_    # -1 = ruído
n_clusters = len(set(db_labels)) - (1 if -1 in db_labels else 0)
print("DBSCAN clusters:", n_clusters, "| Ruído:", (db_labels == -1).sum())

plt.figure()
plt.scatter(X_pca2[:,0], X_pca2[:,1], c=db_labels)
plt.title("DBSCAN sobre PCA2 (-1 = ruído)")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.show()


In [ ]:

from sklearn.pipeline import Pipeline
import joblib

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=2, random_state=42)),
    ("kmeans", KMeans(n_clusters=best_k, n_init=10, random_state=42))
])
pipe.fit(X)

save_path = "/mnt/data/modelo_cap7_kmeans_pca.joblib"
joblib.dump(pipe, save_path)
print("Pipeline salvo em:", save_path)



---
## Conclusões
- **KMeans** precisa de *k*; use **elbow** e **silhouette** como guias (não regras).  
- **DBSCAN** encontra formas arbitrárias e ruído, mas é sensível a `eps/min_samples`.  
- **PCA** ajuda na visualização e pode melhorar separabilidade (cuidado com perda de informação).  
- Salve *pipelines* (scaler+PCA+KMeans) para reproduzir o mesmo *fit* e rótulos.
